In [1]:
import numpy as np
import pandas as pd
import psutil as ps
import os
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from Utility import draw_heatmap, draw_bitmap, draw_importance_forest, cross_val_RF, cross_val_Lin, cross_val_NN
import time

nb_fold = 10
nb_feature = 5
df = pd.read_csv('./data/train.csv').values
train_df, test_df = train_test_split(df, test_size=0.3)
X_train = train_df[:, 2:2+nb_feature]
y_train = train_df[:, -1]
X_test = test_df[:, 2:2+nb_feature]
y_test = test_df[:, -1]
print X_train.shape, y_train.shape
print X_train[:3], y_train[:3]
print X_test.shape, y_test.shape
print X_test[:3], y_test[:3]

Using TensorFlow backend.


(335, 5) (335,)
[[80.47 9.58 82.947 18.112000000000002 19]
 [29.896320000000003 19.75825 24.377 11.978 19]
 [87.30360999999999 6.83939 82.947 18.112000000000002 19]] [-90.9 -104.86666670000001 -96.0]
(144, 5) (144,)
[[9.38 14.13 24.377 11.978 19]
 [71.72 4.26 69.175 11.247 19]
 [27.82902 19.75825 24.377 11.978 19]] [-122.13333329999999 -97.2 -102.66666670000001]


In [4]:
y_pred = searcher.predict(X_test)
#print y_pred[:10]
#print y_test[:10]
print "--- Model--- %s features" %(nb_feature)
print "[MSE]: %.3f" % mean_squared_error(y_test, y_pred)
print "[RMSE]: %.3f" % (mean_squared_error(y_test, y_pred))**(0.5)
print '[R2]: %.3f' % r2_score(y_test, y_pred)

--- Model--- 5 features
[MSE]: 28.884
[RMSE]: 5.374
[R2]: 0.836


In [46]:
#Decision tree
model = DecisionTreeRegressor()
param_grid = {'max_depth': [1, 10, 20, 30, 40, 50]}
searcher = GridSearchCV(estimator=model, param_grid=param_grid, cv=nb_fold, n_jobs=-1)    
searcher.fit(X_train, y_train)  
print searcher.best_score_
print searcher.best_params_

0.758684702701
{'max_depth': 30}


In [118]:
#Random forest
model = RandomForestRegressor()
param_grid = {
    'n_estimators': [1, 50, 100],
    'max_depth': [10, 30, 50],
    'max_features': ['sqrt', 'auto'] 
}
searcher = GridSearchCV(estimator=model, param_grid=param_grid, cv=nb_fold, n_jobs=-1)    
searcher.fit(X_train, y_train)  
print searcher.best_score_
print searcher.best_params_

0.847706700719
{'max_features': 'auto', 'n_estimators': 50, 'max_depth': 50}


In [116]:
#Gradient Boosting
model = GradientBoostingRegressor()
param_grid = {
    'n_estimators': [1, 50, 100],
    'max_depth': [1, 2, 3],
    'learning_rate' : [0.1, 0.01] 
}
searcher = GridSearchCV(estimator=model, param_grid=param_grid, cv=nb_fold, n_jobs=-1)    
searcher.fit(X_train, y_train)  
print searcher.best_score_
print searcher.best_params_

0.842648857024
{'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 5}


In [123]:
# K-NN
model = KNeighborsRegressor()
param_grid = {'n_neighbors': [1, 3, 5, 7, 9]}
searcher = GridSearchCV(estimator=model, param_grid=param_grid, cv=nb_fold, n_jobs=-1)    
searcher.fit(X_train, y_train)  
print searcher.best_score_
print searcher.best_params_

0.809960920268
{'n_neighbors': 3}


In [18]:
#Linear regression
model = linear_model.LinearRegression()
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [2]:
#NN
begin_time = time.time()
def create_model():
    model = Sequential()    
    model.add(Dense(512, activation='relu', input_dim = nb_feature)) 
    model.add(Dense(512, activation='relu'))    
    model.add(Dense(512, activation='relu'))     
    model.add(Dense(512, activation='relu'))   
    model.add(Dense(1))     
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model
model = KerasRegressor(build_fn=create_model, epochs=1000)
param_grid = {'batch_size' : [16, 128]}
searcher = GridSearchCV(estimator=model, param_grid=param_grid, cv=nb_fold, n_jobs=-1)
searcher.fit(X_train, y_train)
print searcher.best_score_
print searcher.best_params_
print time.time() - begin_time

Epoch 1/1000
Epoch 1/1000
301/301 [==============================] - 1s - loss: 2745.0200     
Epoch 2/1000
301/301 [==============================] - 0s - loss: 663.0318     
Epoch 3/1000
224/301 [=====================>........] - ETA: 0s - loss: 358.5089
Epoch 3/1000
 32/301 [==>...........................] - ETA: 1s - loss: 277.2010
Epoch 4/1000
301/301 [==============================] - 1s - loss: 212.0076     
Epoch 5/1000
301/301 [==============================] - 1s - loss: 211.3588     
Epoch 5/1000
 64/301 [=====>........................] - ETA: 1s - loss: 241.0308
Epoch 6/1000
301/301 [==============================] - 1s - loss: 168.8241     
Epoch 6/1000
301/301 [==============================] - 1s - loss: 143.9489     
Epoch 7/1000
256/301 [========================>.....] - ETA: 0s - loss: 153.8438
Epoch 7/1000
301/301 [==============================] - 1s - loss: 154.7049     
Epoch 8/1000
301/301 [==============================] - 1s - loss: 129.6798     
Epoch 9/1000
3

301/301 [==============================] - 1s - loss: 45.2575     
Epoch 45/1000
301/301 [==============================] - 1s - loss: 51.5669     
Epoch 47/1000
301/301 [==============================] - 1s - loss: 47.9205     
Epoch 46/1000
301/301 [==============================] - 1s - loss: 50.4122     
Epoch 48/1000
272/301 [==========================>...] - ETA: 0s - loss: 49.7182
Epoch 49/1000
240/301 [======================>.......] - ETA: 0s - loss: 60.8238
Epoch 50/1000
301/301 [==============================] - 1s - loss: 61.6225     
Epoch 49/1000
301/301 [==============================] - 1s - loss: 73.9640     
Epoch 51/1000
301/301 [==============================] - 1s - loss: 72.9892     
Epoch 50/1000
301/301 [==============================] - 1s - loss: 54.6532     
Epoch 52/1000
301/301 [==============================] - 1s - loss: 48.8246     
Epoch 51/1000
256/301 [========================>.....] - ETA: 0s - loss: 55.6782
Epoch 52/1000
 48/301 [===>...............

301/301 [==============================] - 1s - loss: 55.4473     
Epoch 91/1000
301/301 [==============================] - 1s - loss: 72.4528     
Epoch 91/1000
301/301 [==============================] - 1s - loss: 65.4120     
Epoch 92/1000
 64/301 [=====>........................] - ETA: 0s - loss: 48.0978
Epoch 93/1000
224/301 [=====================>........] - ETA: 0s - loss: 45.6910
Epoch 94/1000
301/301 [==============================] - 1s - loss: 46.9468     
Epoch 94/1000
301/301 [==============================] - 1s - loss: 49.9244     
Epoch 95/1000
128/301 [===========>..................] - ETA: 0s - loss: 80.8053
Epoch 95/1000
301/301 [==============================] - 1s - loss: 84.3778     
Epoch 96/1000
301/301 [==============================] - 1s - loss: 101.7743     
Epoch 96/1000
301/301 [==============================] - 1s - loss: 46.5964     
Epoch 97/1000
301/301 [==============================] - 1s - loss: 46.0711     
Epoch 98/1000
224/301 [==================

301/301 [==============================] - 1s - loss: 48.8992     
Epoch 132/1000
301/301 [==============================] - 1s - loss: 63.6689     
Epoch 133/1000
224/301 [=====================>........] - ETA: 0s - loss: 57.4373
Epoch 135/1000
301/301 [==============================] - 1s - loss: 55.0258     
Epoch 134/1000
301/301 [==============================] - 1s - loss: 40.0513     
Epoch 136/1000
301/301 [==============================] - 1s - loss: 47.9631     
Epoch 135/1000
301/301 [==============================] - 1s - loss: 41.8487     
Epoch 137/1000
112/301 [==========>...................] - ETA: 0s - loss: 45.2128
Epoch 136/1000
301/301 [==============================] - 1s - loss: 48.6511     
Epoch 138/1000
301/301 [==============================] - 1s - loss: 53.7824     
Epoch 137/1000
301/301 [==============================] - 1s - loss: 51.3218     
Epoch 139/1000
301/301 [==============================] - 1s - loss: 55.3723     
Epoch 138/1000
301/301 [=======

208/301 [===================>..........] - ETA: 0s - loss: 58.6341
Epoch 179/1000
301/301 [==============================] - 1s - loss: 57.6007     
Epoch 178/1000
301/301 [==============================] - 1s - loss: 62.8346     
Epoch 180/1000
301/301 [==============================] - 1s - loss: 56.6796     
Epoch 181/1000
144/301 [=============>................] - ETA: 0s - loss: 35.6394
Epoch 180/1000
301/301 [==============================] - 1s - loss: 62.8172     
Epoch 181/1000
112/301 [==========>...................] - ETA: 0s - loss: 41.3977
Epoch 183/1000
224/301 [=====================>........] - ETA: 0s - loss: 51.8107
Epoch 182/1000
 80/301 [======>.......................] - ETA: 0s - loss: 36.3695
Epoch 184/1000
301/301 [==============================] - 1s - loss: 41.3181     
Epoch 183/1000
 80/301 [======>.......................] - ETA: 0s - loss: 37.0250
Epoch 185/1000
301/301 [==============================] - 1s - loss: 45.4512     
Epoch 184/1000
301/301 [=======

...

335/335 [==============================] - 0s - loss: 25.7115     
Epoch 606/1000
335/335 [==============================] - 0s - loss: 24.2945     
Epoch 607/1000
335/335 [==============================] - 0s - loss: 23.7671     
Epoch 608/1000
335/335 [==============================] - 0s - loss: 21.3022     
Epoch 609/1000
335/335 [==============================] - 0s - loss: 19.5627     
Epoch 610/1000
335/335 [==============================] - 0s - loss: 20.7322     
Epoch 611/1000
335/335 [==============================] - 0s - loss: 22.7856     
Epoch 612/1000
335/335 [==============================] - 0s - loss: 22.7122     
Epoch 613/1000
335/335 [==============================] - 0s - loss: 20.6558     
Epoch 614/1000
335/335 [==============================] - 0s - loss: 19.8171     
Epoch 615/1000
335/335 [==============================] - 0s - loss: 27.5392     
Epoch 616/1000
335/335 [==============================] - 0s - loss: 28.8765     
Epoch 617/1000
335/335 [=======

335/335 [==============================] - 0s - loss: 24.4427     
Epoch 706/1000
335/335 [==============================] - 0s - loss: 23.8504     
Epoch 707/1000
335/335 [==============================] - 0s - loss: 26.6037     
Epoch 708/1000
335/335 [==============================] - 0s - loss: 24.9282     
Epoch 709/1000
335/335 [==============================] - 0s - loss: 28.2979     
Epoch 710/1000
335/335 [==============================] - 0s - loss: 25.1205     
Epoch 711/1000
335/335 [==============================] - 0s - loss: 22.4629     
Epoch 712/1000
335/335 [==============================] - 0s - loss: 21.9020     
Epoch 713/1000
335/335 [==============================] - 0s - loss: 19.7212     
Epoch 714/1000
335/335 [==============================] - 0s - loss: 19.7708     
Epoch 715/1000
335/335 [==============================] - 0s - loss: 18.3585     
Epoch 716/1000
335/335 [==============================] - 0s - loss: 17.9245     
Epoch 717/1000
335/335 [=======

335/335 [==============================] - 0s - loss: 18.4897     
Epoch 806/1000
335/335 [==============================] - 0s - loss: 18.4774     
Epoch 807/1000
335/335 [==============================] - 0s - loss: 19.6900     
Epoch 808/1000
335/335 [==============================] - 0s - loss: 19.7711     
Epoch 809/1000
335/335 [==============================] - 0s - loss: 18.8045     
Epoch 810/1000
335/335 [==============================] - 0s - loss: 18.8519     
Epoch 811/1000
335/335 [==============================] - 0s - loss: 20.8725     
Epoch 812/1000
335/335 [==============================] - 0s - loss: 20.3794     
Epoch 813/1000
335/335 [==============================] - 0s - loss: 21.1341     
Epoch 814/1000
335/335 [==============================] - 0s - loss: 23.6388     
Epoch 815/1000
335/335 [==============================] - 0s - loss: 24.5329     
Epoch 816/1000
335/335 [==============================] - 0s - loss: 23.3081     
Epoch 817/1000
335/335 [=======

335/335 [==============================] - 0s - loss: 18.6088     
Epoch 905/1000
335/335 [==============================] - 0s - loss: 19.2524     
Epoch 906/1000
335/335 [==============================] - 0s - loss: 15.3221     
Epoch 907/1000
335/335 [==============================] - 0s - loss: 14.9248     
Epoch 908/1000
335/335 [==============================] - 0s - loss: 16.3053     
Epoch 909/1000
335/335 [==============================] - 0s - loss: 16.4393     
Epoch 910/1000
335/335 [==============================] - 0s - loss: 17.5635     
Epoch 911/1000
335/335 [==============================] - 0s - loss: 17.7087     
Epoch 912/1000
335/335 [==============================] - 0s - loss: 15.6268     
Epoch 913/1000
335/335 [==============================] - 0s - loss: 20.7026     
Epoch 914/1000
335/335 [==============================] - 0s - loss: 17.5077     
Epoch 915/1000
335/335 [==============================] - 0s - loss: 16.5225     
Epoch 916/1000
335/335 [=======

"\nmodel = Sequential()    \nmodel.add(Dense(512, activation='relu', input_dim = nb_feature)) \nmodel.add(Dense(512, activation='relu'))    \nmodel.add(Dense(512, activation='relu'))     \nmodel.add(Dense(512, activation='relu'))   \nmodel.add(Dense(1))     \nmodel.compile(loss='mean_squared_error', optimizer='adam')    \nmodel.fit(X_train, y_train, nb_epoch=1000, batch_size=16, shuffle=True)\n"